###Import libraries and models from the libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_classif
from numpy import array
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
import joblib
from imblearn.over_sampling import SMOTE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/JnU Training/diabetes - diabetes.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
# df = pd.read_csv('/content/diabetes - diabetes.csv')
# df.head()

In [ ]:
df.Outcome.value_counts() # The dataset is imbalanced

,count
Outcome,
0,500
1,268


###Missing value checking

In [ ]:
df.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [ ]:
continuous_feature_names = ['Glucose', 'BloodPressure', 'SkinThickness', 'BMI', 'DiabetesPedigreeFunction', 'Age']
categorical_feature_names = df.drop(columns=continuous_feature_names).columns

In [ ]:
categorical_feature_names

Index(['Pregnancies', 'Insulin', 'Outcome'], dtype='object')

In [ ]:
continuous_features = df[continuous_feature_names]
continuous_features.head()

,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age
0,148,72,35,33.6,0.627,50
1,85,66,29,26.6,0.351,31
2,183,64,0,23.3,0.672,32
3,89,66,23,28.1,0.167,21
4,137,40,35,43.1,2.288,33


In [ ]:
categorical_features = df[categorical_feature_names]
categorical_features.head()

,Pregnancies,Insulin,Outcome
0,6,0,1
1,1,0,0
2,8,0,1
3,1,94,0
4,0,168,1


In [ ]:
scaler = StandardScaler()

In [ ]:
standardized_array = scaler.fit_transform(df[continuous_feature_names])

In [ ]:
continuous_features_scaled = pd.DataFrame(standardized_array, columns=continuous_feature_names)
continuous_features_scaled

,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age
0,0.848324,0.149641,0.907270,0.204013,0.468492,1.425995
1,-1.123396,-0.160546,0.530902,-0.684422,-0.365061,-0.190672
2,1.943724,-0.263941,-1.288212,-1.103255,0.604397,-0.105584
3,-0.998208,-0.160546,0.154533,-0.494043,-0.920763,-1.041549
4,0.504055,-1.504687,0.907270,1.409746,5.484909,-0.020496
...,...,...,...,...,...,...
763,-0.622642,0.356432,1.722735,0.115169,-0.908682,2.532136
764,0.034598,0.046245,0.405445,0.610154,-0.398282,-0.531023
765,0.003301,0.149641,0.154533,-0.735190,-0.685193,-0.275760
766,0.159787,-0.470732,-1.288212,-0.240205,-0.371101,1.170732


In [ ]:
new_df = continuous_features_scaled.join(categorical_features)
new_df.head()

,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Pregnancies,Insulin,Outcome
0,0.848324,0.149641,0.907270,0.204013,0.468492,1.425995,6,0,1
1,-1.123396,-0.160546,0.530902,-0.684422,-0.365061,-0.190672,1,0,0
2,1.943724,-0.263941,-1.288212,-1.103255,0.604397,-0.105584,8,0,1
3,-0.998208,-0.160546,0.154533,-0.494043,-0.920763,-1.041549,1,94,0
4,0.504055,-1.504687,0.907270,1.409746,5.484909,-0.020496,0,168,1


###Feature selection(optional)

In [ ]:
# # Since some values are negative, it should be done positive so that we can use the df for chi2
# final_df = new_df.abs()
# final_df.head()

In [ ]:
# selector = SelectKBest(score_func=chi2, k=7)
# X = selector.fit_transform(final_df.drop(columns=['Outcome'], axis=1), final_df['Outcome'])

In [ ]:
# y = final_df['Outcome']

##KNN

####Hyper Parameter Tuning

In [ ]:
X = new_df.drop(columns=['Outcome'], axis=1)
y = new_df['Outcome']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=56, test_size=0.2)

In [ ]:
parameters = {
    'n_neighbors': [1,2,3,4,5],
    'metric': ['euclidean', 'manhattan']
}

In [ ]:
knn = KNeighborsClassifier()
clf = GridSearchCV(knn, parameters)
clf.fit(X_train, y_train)
print(clf.best_params_)
print(clf.best_estimator_)
print(clf.best_score_)
print("Training score: ", clf.score(X_train, y_train))
print("Testing Score: ", clf.score(X_test, y_test))

{'metric': 'manhattan', 'n_neighbors': 5}
KNeighborsClassifier(metric='manhattan')
0.7013793103448276
Training score:  0.8159722222222222
Testing Score:  0.640625


####Training and testing scores

In [ ]:
#Using the best estimator
knn = KNeighborsClassifier(metric= 'manhattan', n_neighbors= 4)
knn.fit(X_train, y_train)
print("Training score: ", knn.score(X_train, y_train))
print("Testing Score: ", knn.score(X_test, y_test))

Training score:  0.7736156351791531
Testing Score:  0.7597402597402597


####StratifiedKFold Cross Validation

In [ ]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
accuracy_scores = []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = np.array(X)[train_index], np.array(X)[test_index] # X should be numpy array, not dataframe
    y_train, y_test = y[train_index], y[test_index]

    knn.fit(X_train, y_train)

    y_pred = knn.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

print("Accuracy scores for each fold:", accuracy_scores)
print("Mean accuracy:", sum(accuracy_scores) / len(accuracy_scores))

Accuracy scores for each fold: [0.6948051948051948, 0.6753246753246753, 0.6948051948051948, 0.6928104575163399, 0.6797385620915033]
Mean accuracy: 0.6874968169085817


##GaussianNB

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=56, test_size=0.2)

####Hyperparameter Tuining

In [ ]:
param_grid = {
    'var_smoothing': [1e-10, 1e-9, 1e-8, 1e-7, 1e-6]
}

gnb = GaussianNB()

stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=gnb,
    param_grid=param_grid,
    cv=stratified_kfold,
    scoring='accuracy'
)

grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best cross-validation accuracy score:", grid_search.best_score_)

Best parameters: {'var_smoothing': 1e-06}
Best cross-validation accuracy score: 0.7492336398773823


In [ ]:
gnb = GaussianNB(var_smoothing = 1e-06)
gnb.fit(X_train, y_train)

GaussianNB(var_smoothing=1e-06)

####Training and testing scores

In [ ]:
print(gnb.score(X_train, y_train))
print(gnb.score(X_test, y_test))

0.7638436482084691
0.7922077922077922


####StratifiedKFold Cross Validation

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

model = GaussianNB(var_smoothing = 1e-06)
accuracy_scores = []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = np.array(X)[train_index], np.array(X)[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

print("Accuracy scores for each fold:", accuracy_scores)
print("Mean accuracy:", np.mean(accuracy_scores))

Accuracy scores for each fold: [0.7207792207792207, 0.8116883116883117, 0.7597402597402597, 0.7320261437908496, 0.738562091503268]
Mean accuracy: 0.752559205500382


##Deployment

In [ ]:
joblib.dump(gnb, 'gnb.pkl')

['gnb.pkl']